In [17]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.sql("""select distinct(Account_Number), 
Capital_Disp_Inv_Item, 
Product_Segment_Extnl_Cap_Disp,
ShiptoState, 
Ordered_Date, 
Resolved_Price_USD, 
Major_Description, 
Minor_Description, 
Item_Description, 
Customer_class_code,
DescriptionC ,
Round(Quantity_Ordered,1) AS Quantity_Ordered,
DescriptionF 
from KPMG_Bronze.dbo.EDW_Conmed_Sales_Data_Mart """)
display(df)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b8380583-e11c-4242-9c46-fa4d9c4a987b)

In [18]:
from pyspark.sql.functions import col, max, count, sum, datediff, current_date

# Convert Order_Date to date type
df = df.withColumn("Ordered_Date", col("Ordered_Date").cast("date"))

# Display sample
display(df)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7711529a-7f6f-494c-9201-90b54b3a6f08)

In [19]:
from pyspark.sql.functions import to_date, lit, datediff, max, count, sum, col, year

# Define the fixed date for 2023
fixed_date = to_date(lit('2024-12-31'))

# **Filter data for only 2023**
df_2020 = df.filter(year(col("Ordered_Date")) == 2024)

# Calculate RFM values for 2023
rfm_df = df_2020.groupBy("Account_Number").agg(
    datediff(fixed_date, max("Ordered_Date")).alias("Recency"),
    count("Quantity_Ordered").alias("Frequency"),
    sum(col("Resolved_Price_USD") * col("Quantity_Ordered")).alias("Monetary")
)

# Join RFM values back to the 2023 DataFrame
df_2020 = df_2020.join(rfm_df, on="Account_Number", how="left")

# Display the updated DataFrame (for 2023 only)
display(df_2020)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6c43e2cd-995b-4104-8b42-2e685889befb)

In [20]:
from pyspark.sql.functions import ntile
from pyspark.sql.window import Window


# Rank customers into 5 bins (1 = worst, 5 = best)
df_2020 = df_2020.withColumn("R_Score", ntile(5).over(Window.orderBy("Recency")))
df_2020 = df_2020.withColumn("F_Score", ntile(5).over(Window.orderBy(col("Frequency").desc())))
df_2020 = df_2020.withColumn("M_Score", ntile(5).over(Window.orderBy(col("Monetary").desc())))

# Create RFM Score as a string
df_2020 = df_2020.withColumn("RFM_Score", col("R_Score") + col("F_Score") + col("M_Score"))

# Show results
display(df_2020)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0796be72-e992-49aa-ad40-ef5cc18e527b)

In [21]:
from pyspark.sql.functions import when

df_2020 = df_2020.withColumn("Segment",
    when(col("RFM_Score") >= 13, "VIP Customer")
    .when((col("RFM_Score") >= 10) & (col("RFM_Score") < 13), "Loyal Customer")
    .when((col("RFM_Score") >= 6) & (col("RFM_Score") < 10), "At-Risk Customer")
    .otherwise("Lost Customer")
)

display(df_2020)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8e745e0f-30db-4923-a057-cf88323e344a)

In [22]:
import pandas as pd
import plotly.express as px

    # Convert PySpark DataFrame to Pandas
rfm_pandas = df_2020.select("Segment").toPandas()

    # Count the number of customers in each segment
rfm_counts = rfm_pandas["Segment"].value_counts().reset_index()
rfm_counts.columns = ["Segment", "Count"]

    # Plot the chart
fig = px.bar(
    rfm_counts,
    x="Segment",
    y="Count",
    color="Segment",
    title="Customer Segmentation Distribution",
    text_auto=True
    )

fig.show()


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 24, Finished, Available, Finished)

In [23]:
from pyspark.sql.functions import col

# Convert RFM_Score to double if needed
df_2020 = df_2020.withColumn("RFM_Score", col("RFM_Score").cast("double"))
df_2020 = df_2020.withColumn("Recency", col("Recency").cast("double"))
df_2020 = df_2020.withColumn("Frequency", col("Frequency").cast("double"))
df_2020 = df_2020.withColumn("Monetary", col("Monetary").cast("double"))

df_2020.printSchema()  # Check data types


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 25, Finished, Available, Finished)

root
 |-- Account_Number: string (nullable = true)
 |-- Capital_Disp_Inv_Item: string (nullable = true)
 |-- Product_Segment_Extnl_Cap_Disp: string (nullable = true)
 |-- ShiptoState: string (nullable = true)
 |-- Ordered_Date: date (nullable = true)
 |-- Resolved_Price_USD: double (nullable = true)
 |-- Major_Description: string (nullable = true)
 |-- Minor_Description: string (nullable = true)
 |-- Item_Description: string (nullable = true)
 |-- Customer_class_code: string (nullable = true)
 |-- DescriptionC: string (nullable = true)
 |-- Quantity_Ordered: decimal(22,1) (nullable = true)
 |-- DescriptionF: string (nullable = true)
 |-- Recency: double (nullable = true)
 |-- Frequency: double (nullable = true)
 |-- Monetary: double (nullable = true)
 |-- R_Score: integer (nullable = false)
 |-- F_Score: integer (nullable = false)
 |-- M_Score: integer (nullable = false)
 |-- RFM_Score: double (nullable = false)
 |-- Segment: string (nullable = false)



In [24]:
from pyspark.ml.feature import VectorAssembler

# Assemble numeric features into a single vector column
assembler = VectorAssembler(
    inputCols=["Recency", "Frequency", "Monetary"], 
    outputCol="features",
    handleInvalid="skip"  # Drop rows with invalid data
)

rfm_df_transformed = assembler.transform(df_2020)
rfm_df_transformed.select("features").show(5, truncate=False)  # Check transformation



StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 26, Finished, Available, Finished)

+-------------------------------+
|features                       |
+-------------------------------+
|[13.0,289.0,181749.02990000002]|
|[13.0,289.0,181749.02990000002]|
|[13.0,289.0,181749.02990000002]|
|[13.0,289.0,181749.02990000002]|
|[13.0,289.0,181749.02990000002]|
+-------------------------------+
only showing top 5 rows



In [25]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(rfm_df_transformed)
rfm_scaled = scaler_model.transform(rfm_df_transformed)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 27, Finished, Available, Finished)

In [26]:
from pyspark.ml.clustering import KMeans

# Train K-Means model
kmeans = KMeans(featuresCol="scaled_features", k=5, seed=42)
model = kmeans.fit(rfm_scaled)

# Assign clusters to customers
rfm_clusters = model.transform(rfm_scaled)

# Show results
display(rfm_clusters.select("Recency", "Frequency", "Monetary", "prediction"))


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b65d1a21-30fe-4ec7-ba7a-95fa791876a4)

In [27]:
from pyspark.sql.functions import col

rfm_clusters = rfm_clusters.withColumn("KMeans_Segment",
    when(col("prediction") == 0, "At-Risk Customers") 
    .when(col("prediction") == 1, "Regular Customers") 
    .when(col("prediction") == 2, "Premium Customers") 
    .when(col("prediction") == 3, "Loyal Customer")
    .when(col("prediction") == 4, "casual customer") 
)

rfm_clusters.select("Account_Number", "Recency", "Frequency", "Monetary", "prediction", "KMeans_Segment").show(10)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 29, Finished, Available, Finished)

+--------------+-------+---------+------------------+----------+-----------------+
|Account_Number|Recency|Frequency|          Monetary|prediction|   KMeans_Segment|
+--------------+-------+---------+------------------+----------+-----------------+
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|          9009|   13.0|    289.0|181749.02990000002|         1|Regular Customers|
|   

In [28]:
rfm_clusters.select("Account_Number").distinct().count()

StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 30, Finished, Available, Finished)

6163

In [29]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
# Count the number of customers per CLTV segment
segment_counts = rfm_clusters.groupBy("KMeans_Segment").agg(
    F.count("Account_Number").alias("Customer_Count")
)

segment_counts.orderBy("KMeans_Segment").show()


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 31, Finished, Available, Finished)

+-----------------+--------------+
|   KMeans_Segment|Customer_Count|
+-----------------+--------------+
|At-Risk Customers|         36797|
|   Loyal Customer|           675|
|Premium Customers|          5350|
|Regular Customers|        227713|
|  casual customer|         33668|
+-----------------+--------------+



In [30]:
from pyspark.sql.functions import avg, col, when

# Calculate average RFM values per cluster
cluster_avg = rfm_clusters.groupBy("prediction").agg(
    avg("Recency").alias("Avg_Recency"),
    avg("Frequency").alias("Avg_Frequency"),
    avg("Monetary").alias("Avg_Monetary")
)

# Assign segment names based on cluster prediction
cluster_avg = cluster_avg.withColumn(
    "KMeans_Segment",
    when(col("prediction") == 0, "At-Risk Customers") 
    .when(col("prediction") == 1, "Regular Customers") 
    .when(col("prediction") == 2, "Premium Customers") 
    .when(col("prediction") == 3, "Loyal Customer") 
    .when(col("prediction") == 4, "Casual Customer") 
)

# Show results
display(cluster_avg)


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ef08260e-8a60-4a71-af90-0ca53370ef0c)

In [31]:
from pyspark.sql import functions as F

aggregated_info = rfm_clusters.groupBy("prediction").agg(
    F.collect_set("Major_Description").alias("Distinct_Products"),
    F.collect_set("DESCRIPTIONF").alias("Distinct_sales_Rep"),
    F.collect_set("DESCRIPTIONC").alias("Distinct_DESCRIPTIONC"),
    F.collect_set("Capital_Disp_Inv_Item").alias("Distinct_Capital_Disp_Inv_Item")
)

display(aggregated_info.orderBy("prediction"))


StatementMeta(, 100f370b-1e86-4ddb-8644-4218ca65c2a7, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a54e3460-5f51-41cc-8761-1fce413e3c4d)